# Στοχαστικές Διεργασίες και Βελτιστοποίηση στη Μηχανική Μάθηση

## 12ο Εργαστήριο - *LSTM's - DGA Algorithm*

- Ονομ/νυμο: Χρήστος Νίκου
- AM: 03400146
- Ιδιότητα: Μεταπτυχιακός φοιτητής Επιστήμης Δεδομένων και Μηχανικής Μάθησης (ΕΔΕΜΜ)
- Ηλεκτρονική Διεύθυνση: christosnikou@mail.ntua.gr / chrisnick92@gmail.com

Πρόσφατες επιθέσεις, όπως εκείνη στον πάροχο DNS Dyn (<a href="https://en.wikipedia.org/wiki/DDoS_attack_on_Dyn">wiki</a>) αποδεικνύουν ότι ένα από τα σημαντικότερα προβλήματα που αντιμετωπίζει το σύγχρονο Διαδίκτυο είναι εκείνο των botnets. Σε αυτά, ένας επιτιθέμενος συγκεντρώνει την υπολογιστική ισχύ που του είναι απαραίτητη για την εκδήλωση επιθέσεων DDoS ή/και άλλων κακόβουλων δραστηριοτήτων εγκαθιστώντας λογισμικό σε μεγάλο πλήθος από υπολογιστές (bots) που έχουν κενά ασφαλείας (π.χ. συσκευές Internet of Things - IoT).

Οι μολυσμένοι υπολογιστές (bots) διατηρούν διαύλους επικοινωνίας με το διαχειριστή του botnet (Command & Control Server) με σκοπό να λαμβάνουν εντολές και να αποστέλλουν πληροφορίες. Για το σκοπό αυτό εκμεταλλεύονται καθιερωμένα πρωτόκολλα, όπως το DNS με την παραγωγή μεγάλου πλήθους από domain names μέσω Domain Generation Algorithms (DGA's) που αλλάζουν διαρκώς για την επικοινωνία του bot με το διαχειριστή του, ώστε να αποφεύγεται ο εντοπισμός του Command & Control Server.

Τα ονόματα DNS που χρησιμοποιούνται από αλγορίθμους DGA μπορεί να είναι είτε τυχαία αλφαριθμητικά (π.χ. asdfasjkdfh8oawher8has.com) ή συνδυασμοί τυχαίων λέξεων που έχουν ληφθεί από κάποιο λεξικό (π.χ. school-doctor.com). Χρησιμοποιείται ένας μεγάλος αριθμός από τέτοια ονόματα, η πλειοψηφία των οποίων δεν έχουν κάποια αντιστοίχιση σε διεύθυνση IP και στοχεύουν στην απόκρυψη του Command & Control Server, επειδή οι αμυνόμενοι καλούνται να ελέγξουν κάθε ένα από τα ονόματα που παρατηρούν στο δίκτυό τους, σπαταλώντας χρόνο και πόρους. Επιπρόσθετα, η διεύθυνση IP του Command & Control Server αλλάζει πολύ συχνά (πολλές φορές σε μία μέρα), ώστε να αποφεύγεται ο εντοπισμός του ακόμα και όταν εντοπίζονται τα ονόματα DGA που οδήγησαν σε αυτόν.

<img src="https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab12/dga.png"></img>

In [ ]:
!wget https://raw.githubusercontent.com/ChrisNick92/StochasticsLabPublic/master/lab12/dga_domains_full.csv

--2022-07-07 20:00:06--  https://raw.githubusercontent.com/ChrisNick92/StochasticsLabPublic/master/lab12/dga_domains_full.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19952823 (19M) [text/plain]
Saving to: ‘dga_domains_full.csv’

dga_domains_full.cs 100%[===================>]  19.03M  --.-KB/s    in 0.08s   

2022-07-07 20:00:06 (250 MB/s) - ‘dga_domains_full.csv’ saved [19952823/19952823]



<img src=""></img>

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import random


def add_word_distinct_chars(string, distinct_chars):
    for char in string:
        distinct_chars.add(char)
    return distinct_chars

def find_max_len(string, max_len):
    string_length = len(string)
    if string_length > max_len:
        max_len = string_length
    return max_len

def load_data(filename):
    dataset = []
    distinct_chars = set()
    max_len = 0
    currentIndex = 0
    diverse_labels = dict()
    with open(filename, "r") as fdr:
        for line in fdr:
            line = line.strip()
            general, label, name = line.split(",")
            name = name.split(".")[0]
            if label not in diverse_labels.keys():
                diverse_labels[label] = currentIndex
                currentIndex += 1
            distinct_chars = add_word_distinct_chars(name, distinct_chars)
            max_len = find_max_len(name, max_len)
            temp_list = []
            temp_list.append(name)
            temp_list.append(label)
            dataset.append(temp_list)
    random.shuffle(dataset)
    return dataset, distinct_chars, max_len, diverse_labels

def assign_index(chars):
    features = {}
    for index, char in enumerate(chars):
        features[char] = index
    return features

def convert_dataset_and_tokenize(dataset, features, max_len):
    for item_no, example in enumerate(dataset):
        name = example[0]
        label = example[1]
        tokenized = []
        padding_needed = max_len - len(name)
        for index in range(padding_needed):
            tokenized.append(0)
        for char in name:
            token = features[char]
            tokenized.append(token)
        example[0] = tokenized
        dataset[item_no] = example
    return dataset

def split_examples_labels(dataset):
    examples = [entry[0] for entry in dataset]
    labels = [entry[1] for entry in dataset]
    return examples, labels

def convert_labels_to_int(labels, diverse_labels):
    for index, label in enumerate(labels):
        labels[index] = diverse_labels[label]
    return labels

def build_model(max_features, max_len):
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length = max_len))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(26, activation = 'softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

def return_fold(examples, labels_int, num_folds, current_fold):
    interval = len(examples) // num_folds
    current_start = (current_fold - 1) * interval
    current_end = current_fold * interval
    X_test = examples[current_start:current_end]
    y_test = labels_int[current_start:current_end]
    X_train = examples[:current_start]
    y_train = labels_int[:current_start]
    X_train.extend(examples[current_end:])
    y_train.extend(labels_int[current_end:])
    return X_train, y_train, X_test, y_test

def make_predictions(model, X_test):
    y_pred = model.predict(X_test)
    y_pred = numpy.argmax(y_pred, axis = 1)
    return y_pred

def return_confusion_matrix(y_test, y_pred):
    return confusion_matrix(y_test, y_pred)

def return_classification_report(y_test, y_pred, diverse_names):
    target_names = []
    for item in diverse_names.keys():
        target_names.append(item)
    print(classification_report(y_test, y_pred, target_names = target_names))
    return None
    
if __name__ == "__main__":
    dataset, distinct_chars, max_len, diverse_labels = load_data("dga_domains_full.csv")
    dataset = dataset[0:100000]
    max_features = len(distinct_chars) + 1
    features = assign_index(distinct_chars)
    dataset = convert_dataset_and_tokenize(dataset, features, max_len)
    examples, labels = split_examples_labels(dataset)
    labels_int = convert_labels_to_int(labels, diverse_labels)

    acc_per_fold = []
    loss_per_fold = []
    number_of_folds = 5
    for fold in range(1, number_of_folds + 1):
        X_train, y_train, X_test, y_test = return_fold(examples, labels_int, number_of_folds, fold)
        model = build_model(max_features, max_len)
        print("Training for fold: ", fold)
        history = model.fit(X_train, y_train, batch_size = 128, epochs = 5, verbose = 1)
        scores = model.evaluate(X_test, y_test, verbose = 1)
        print(f'Score for fold {fold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        y_pred = make_predictions(model, X_test)
        print(return_confusion_matrix(y_test, y_pred))
        return_classification_report(y_test, y_pred, diverse_labels)

    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print('------------------------------------------------------------------------')
        print('Average scores for all folds:')
        print(f'> Accuracy: {numpy.mean(acc_per_fold)} (+- {numpy.std(acc_per_fold)})')
        print(f'> Loss: {numpy.mean(loss_per_fold)}')
        print('------------------------------------------------------------------------')


Training for fold:  1
Epoch 1/5
625/625 [==============================] - 180s 284ms/step - loss: 1.4637 - accuracy: 0.6054
Epoch 2/5
625/625 [==============================] - 178s 284ms/step - loss: 0.9636 - accuracy: 0.7070
Epoch 3/5
625/625 [==============================] - 176s 281ms/step - loss: 0.8145 - accuracy: 0.7479
Epoch 4/5
625/625 [==============================] - 178s 285ms/step - loss: 0.7358 - accuracy: 0.7677
Epoch 5/5
625/625 [==============================] - 16s 26ms/step - loss: 0.5682 - accuracy: 0.8171
Score for fold 1: loss of 0.56817227602005; accuracy of 81.71499967575073%
[[ 300    0  113    0    4    0    0    8    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   0  376    0    0    0    0    2    0    0    0    0    0    0    0
     0    0    0    0    2    0    0    0    0   16    0    1]
 [ 109    3 9637    6   65    6    7   40   33   20   33    5   21    1
    59    6   11    1   23    0   31   21    1  

Ο παραπάνω κώδικας επιλύει το πρόβλημα της ταξινόμησης ονομάτων DNS ως προς το αν αυτά τα ονόματα είναι καλόβουλα ή έχουν παραχθεί από Domain Generation Algorithms (DGA's). Συγκεκριμένα, ο κώδικας επιλύει ένα πρόβλημα multi-class classification, όπου τα ονόματα αντιστοιχίζονται είτε στην κατηγορία alexa, εάν είναι καλόβουλα ή στην αντίστοιχα κατηγορία malware στην οποία ανήκουν. Το μοντέλο που έχει χρησιμοποιηθεί για την ταξινόμηση των ονομάτων είναι το LSTM, ενώ για το κατάλληλο validation του μοντέλου έχει χρησιμοποιηθεί η μέθοδος K-fold cross validation με Κ = 5. Τα δεδομένα εκπαίδευσης και αξιολόγησης έχουν ληφθεί από <a href="https://github.com/chrmor/DGA_domains_dataset">εδώ</a> και χρησιμοποιούνται τα πρώτα 100000 ονόματα του αρχείου "dga_domains_full.csv". 

Μελετώντας τον κώδικα και εκτελώντας τον, να απαντήσετε στις ακόλουθες ερωτήσεις:
<ul>
<li>Γιατί το LSTM είναι κατάλληλο μοντέλο για την επίλυση του συγκεκριμένου προβλήματος;</li>"
<li>Τι είναι η λίστα ονομάτων Alexa (<a href="https://en.wikipedia.org/wiki/Alexa_Internet">info</a>), ονόματα της οποίας έχουν χρησιμοποιηθεί για την κατηγορία των καλόβουλων ονομάτων;</li>
<li>Να διαλέξετε δύο οικογένειες malware από την κατηγορία dga, δηλαδή την κατηγορία με τα κακόβουλα ονόματα που αντιστοιχούν σε malware. Να αναζητήσετε τι προβλήματα δημιουργούν τα malware αυτά, π.χ. υποκλοπή τραπεζικών κωδικών, έναρξη επιθέσεων DDoS, κλπ.</li>
<li>Να περιγράψετε σύντομα τα βήματα που ακολουθεί το παραπάνω πρόγραμμα για την επίλυση του προβλήματος.</li>
<li>Ποιος είναι ο ρόλος του embedding layer και τι παραμέτρους δέχεται;</li>
<li>Να αναφέρετε λόγους για τους οποίους χρησιμοποιείται η μέθοδος K-fold cross validation.</li>
<li>Κατά τη χρήση της μεθόδου K-fold cross validation, ανά fold, να αναφέρετε πόσα examples χρησιμοποιούνται για την εκπαίδευση του μοντέλου και πόσα για το validation/testing.
<li>Δείτε <a href="https://en.wikipedia.org/wiki/Cross-validation_(statistics)">εδώ</a> μεθόδους που μπορεί να χρησιμοποιούν έναντι της μεθόδου K-fold cross validation. Να αναφέρετε μερικά πλεονεκτήματα και μειονεκτήματά τους.</li>
<li>Να αναλύσετε τα κριτήρια precision, recall και F1-score που εμφανίζονται στο classification report.</li>
<li>Σε ποιες κατηγορίες πετυχαίνουμε καλύτερα αποτελέσματα και σε ποιες χειρότερα; Γιατί πιστεύετε ότι παίρνουμε αυτά τα αποτελέσματα;</li>
</ul>

### Απαντήσεις

#### Ερώτηση 1

>*Γιατί το LSTM είναι κατάλληλο μοντέλο για την επίλυση του συγκεκριμένου προβλήματος;*

Στο συγκεκριμένο πρόβλημα τα ονόματα των ιστοσελίδων μπορούν να θεωρηθούν ως μια ακολουθία χαρακτήρων. Συνεπώς, η αρχιτεκτονική του *LSTM* είναι η ιδανική για προβλήματα όπου τα δεδομένα εισόδου έχουν ακολουθιακή συμπεριφορά. Όπως βλέπουμε και στα παρακάτω παραδείγματα φαίνεται να υπάρχει κάποια συσχέτιση μεταξύ των λέξεων και γραμμάτων που εμφανίζονται στα ονόματα που προέρχονται από καλόβουλες ιστοσελίδες.

In [ ]:
dataset, distinct_chars, max_len, diverse_labels = load_data("dga_domains_full.csv")
for i,sample in enumerate(dataset[:10]):
  print(f"- Sample:{i+1}{5*' '}domain name:{sample[0]}{5*' '}Type:{sample[1]}")

- Sample:1     domain name:glstvqjcdvhy     Type:necurs
- Sample:2     domain name:fatlockcloudlineclubdiscount     Type:matsnu
- Sample:3     domain name:i65l58m49a57gqiufriyatl48itfzcvezkycz     Type:murofet
- Sample:4     domain name:y6sh14wbcfwxev3baxmd367     Type:corebot
- Sample:5     domain name:kirdehei     Type:pushdo
- Sample:6     domain name:mrunal     Type:alexa
- Sample:7     domain name:justifacts     Type:alexa
- Sample:8     domain name:e8u4u0ysm0m0     Type:qadars
- Sample:9     domain name:plgswdcmvvmpclkko     Type:ranbyus
- Sample:10     domain name:familijny     Type:alexa


Όπως βλέπουμε τα ονόματα που αντιστοιχούν σε καλόβουλες ιστοσελίδες (alexa) μπορούν να αναγνωστούν από έναν άνθρωπο σε αντίθεση με τα ονόματα που αντιστοιχούν σε κακόβουλες ιστοσελίδες. Έτσι, μια αρχιτεκτονική *LSTM* θα μπορέσει να αποκωδικοποιήσει αυτή τη συσχέτιση που υπάρχει μεταξύ των ονομάτων και να διακρίνει τις διαφορετικές κατηγορίες. Επίσης, όπως φαίνεται και παρακάτω υπάρχουν λέξεις που περιέχουν εως και 64 γράμματα πράγμα το οποίο σημαίνει ότι μπορεί να υπάρχει εξάρτηση μεταξύ των γραμματών για μεγάλα χρονικά διαστήματα εντός της ακολουθίας. Έτσι, ένα *LSTM* με τις επιπλέον πύλες (*gates*) που διαθέτει θα μπορέσει να διατηρήσει τη χρήσιμη πληροφορία για μεγαλύτερα χρονικά βήματα αφαιρώντας όλη την πληροφορία που δεν χρειάζεται.

In [ ]:
print(f"- Longest name has {max_len} words.")

- Longest name has 63 words.


#### Ερώτηση 2

>*Τι είναι η λίστα ονομάτων Alexa (<a href="https://en.wikipedia.org/wiki/Alexa_Internet">info</a>), ονόματα της οποίας έχουν χρησιμοποιηθεί για την κατηγορία των καλόβουλων ονομάτων;*

Η λίστα ονομάτων *Alexa* οφείλεται στην εταιρεία *Alexa Internet Inc.* η οποία ήταν Αμερικάνικη εταιρεία που ασχολούνταν με την ανάλυση αναζήτησης ιστοσελίδων στο διαδίκτυο (*web traffic analysis*). Από το έτος ίδρυσης της 1996 εώς και το 1999 η εταιρεία λειτουργούσε ως αυτόνομη εταιρεία ενώ μετά το 1999 αγοράστηκε απ' την *Amazon*. Η εταιρεία αρχικά μέσω μιας εργαλειοθήκης που διέθετε έδινε προτάσεις στους χρήστες της για το ποιες ιστοσελίδες να επισκεφθούν ανάλογα με τα ενδιαφεροντά τους. Τέλος, λόγω του μεγάλου χρηστών που εξυπηρετούσε διέθετε έναν μεγάλο όγκο δεδομένων σχετικά με το ποιες ιστοσελίδες προτιμούσαν να επισκέφτονται οι χρήστες και έτσι μπορούσε να διεξάγει σημαντικές έρευνες για τις προτιμήσεις των χρηστών.

#### Ερώτηση 3

>*Να διαλέξετε δύο οικογένειες malware από την κατηγορία dga, δηλαδή την κατηγορία με τα κακόβουλα ονόματα που αντιστοιχούν σε malware. Να αναζητήσετε τι προβλήματα δημιουργούν τα malware αυτά, π.χ. υποκλοπή τραπεζικών κωδικών, έναρξη επιθέσεων DDoS, κλπ.*

Παρακάτω βλέπουμε τις διάφορες κατηγορίες των *malware* που υπάρχουν στο *csv* αρχείο.

In [ ]:
classes = set()
for sample in dataset:
  if sample[1] != "alexa":
    classes.add(sample[1])

In [ ]:
classes

{'conficker',
 'corebot',
 'cryptolocker',
 'dircrypt',
 'emotet',
 'fobber',
 'gozi',
 'kraken',
 'matsnu',
 'murofet',
 'necurs',
 'nymaim',
 'padcrypt',
 'pushdo',
 'pykspa',
 'qadars',
 'ramdo',
 'ramnit',
 'ranbyus',
 'rovnix',
 'simda',
 'suppobox',
 'symmi',
 'tinba',
 'vawtrak'}

- <b>Cryptolocker:</b> Το *cryptolocker* είναι ένα είδος ιού τύπου *ransomware* ο οποίος κρυπτογραφεί διάφορους φακέλους στον υπολογιστή που έχει εγκατασταθεί και για την αποκρυπτογράφησή τους ζητά από τους χρήστες κάποια πληρωμή έτσι ώστε να αποκτήσουν ξανά πρόσβαση στους μολυσμένους φακέλους. Κατά τη διάρκεια μιας *cryptolocker* επίθεσης ο ιός φτάνει στον υπολογιστή μέσω ενός *e-mail* ή κάποιου *spam* μηνύματος το οποίο περιέχει επιβλαβλή αρχεία ή κάποιο *link* σε μια επιβλαβλής ιστοσελίδα. Όταν ανοιχτεί το αρχείο ο ιός κρυπτογραφεί ένα μεγάλο μέρος των τοπικών φακέλων του υπολογιστή ζητώντας πληρωμή ως αντάλλαγμα για την αποκρυπτογράφηση των φακέλων.

- <b> Symmi: </b>Ο *symmi* είναι ένας είδος ιού τύπου *trojan*. Ο *symmi* περνάει μέσα στον υπολογιστή μέσω ενός μολυσμένου φακέλου ο οποίος μπορεί να έχει βρεθεί στον υπολογιστή με διάφορους τρόπους (π.χ. email, spam messages, links κτλ). Ο ιός αυτός μπορεί να παίξει τον ρόλο του διαμεσολαβητή για την εγκατάσταση άλλων ιών. Παρ'όλα αυτά, ένας τέτοιος ιός μπορεί τραβήξει *screenshots* την επιφάνεια εργασίας του κάθε χρήστη, να καταγράψει τις λέξεις που πατήθηκαν μέσω του πληκτρολογίου, να αντιγράψει κωδικούς και να ελέγξει την κάμερα του υπολογιστή.

#### Ερώτηση 4

>*Να περιγράψετε σύντομα τα βήματα που ακολουθεί το παραπάνω πρόγραμμα για την επίλυση του προβλήματος.*

Η εκτέλεση του προγράμματος ξεκινάει στο σημείο που ορίζεται η `main`. Αρχικά μέσω της συνάρτησης `load_data` γίνεται η ανάγνωση του αρχείου *csv* και διαμορφώνονται τα δεδομένα. Η συνάρτηση επιστρέφει μια λίστα που περιέχει το όνομα κάθε σελίδας αλλά και την κατηγορία της σελίδας. Επίσης, επιστρέφει το πλήθος των διαφορετικών χαρακτήρων που εμφανίζονται αλλά και το μήκος της μεγαλύτερης ονομασίας. Εν συνεχεία, μέσω της συνάρτησης `assign_index` γίνεται η αντιστοίχιση των μοναδικών χαρακτήρων σε αριθμούς. Ύστερα, μέσω της συνάρτησης `convert_dataset_and_tokenize` γίνεται η μετατροπή της κάθε ονομασίας σε ένα διάνυσμα που περιέχει τους αριθμούς του κάθε γράμματος που εμφανίζεται στην ονομασία. Επίσης, στην αρχή αυτού του διανύσματος προστίθενται μηδενικά μέχρις ότου το διάνυσμα να φτάσει τη διάσταση του διανύσματος που αντιστοιχεί στη μεγαλύτερη λέξη (64 στο συγκεκριμένο παράδειγμα). Μετά χωρίζονται τα labels από τα features και αρχικοποιείται το μοντέλο τύπου *LSTM*. Η διαδικασία του training έχει ως εξής: Το dataset χωρίζεται σε 5 ισοπληθή κομμάτια και καθένα απ'τα 5 κομμάτια παίζει το ρόλο του συνόλου επικύρωσης. Σε κάθε επανάληψη το μοντέλο εκπαιδεύεται στα 4 κομμάτια του dataset για 5 εποχές και αξιολογείται η επίδοσή του στο σύνολο επικύρωσης.

#### Ερώτηση 5

>*Ποιος είναι ο ρόλος του embedding layer και τι παραμέτρους δέχεται;*

Ο ρόλος του *embedding layer* είναι να μετατρέψει τα διανύσματα ακεραίων που δημιουργήθηκαν μέσω της συνάρτησης `convert_dataset_and_tokenize` σε διανύσματα πραγματικών αριθμών αφήνοντας στον σχεδιαστή τη δυνατότητα να επιλέξει τη διάσταση αυτών των διανυσμάτων. Επίσης, με τον τρόπο που λειτουργεί το *embedding layer* τείνει να απεικονίζει λέξεις που έχουν παρόμοια σημασιολογική σε αριθμούς που απέχουν μικρή απόσταση μεταξύ τους κάτι το οποίο δεν συνέβαινε με την αναπαράσταση που είχαμε προηγουμένως. Οι παράμετροι που δέχεται είναι: 1) το πλήθος των διαφορετικών χαρακτηριστικών που έχουμε στο συγκεκριμένο πρόβλημα (`max_features`), το μέγεθος της διάστασης που θέλουμε να απεικονιστούν τα διανύσματα (128 στην περίπτωσή μας) και τέλος το μέγεθος των γραμμάτων που περιέχει η μεγαλύτερη λέξη (`max_len`).

#### Ερώτηση 6

>*Να αναφέρετε λόγους για τους οποίους χρησιμοποιείται η μέθοδος K-fold cross validation.*

Η μέθοδος του *K-fold cross validation* χρησιμοποιείται έτσι ώστε να αποφεχθεί το *overfitting* του μοντέλου πάνω στα δεδομένα εκπαίδευσης. Αυτό συμβαίνει διότι το μοντέλο εκπαιδεύεται σε διαφορετικά μέρη του συνόλου δεδομένων με αποτέλεσμα να μην του επιτρέπεται να απομνημονεύσει ποτέ τον θόρυβο που υπάρχει στα δεδομένα. Ένας άλλος εξίσου σημαντικός λόγος είναι ότι η μέθοδος του *K-fold cross validation* δίνει την δυνατότητα στον σχεδιαστή να αξιολογήσει με μεγαλύτερη ακρίβεια τη γενίκευση του μοντέλου και αυτό διότι κάθε υποσύνολο του συνόλου δεδομένων αποτελεί ένα σύνολο επικύρωσης και έτσι η τελική αξιολόγηση του μοντέλου προκύπτει από ένα όσο το δυνατόν πιο ποικιλόμορφο σύνολο δεδομένων γίνεται.

#### Ερώτηση 7

>*Κατά τη χρήση της μεθόδου K-fold cross validation, ανά fold, να αναφέρετε πόσα examples χρησιμοποιούνται για την εκπαίδευση του μοντέλου και πόσα για το validation/testing.*

Όπως φαίνεται και παρακατώ το αρχικό dataset αποτελείται από 100000 δείγματα επομένως κατά τη διάρκεια μιας επανάληψης του *cross validation* χρησιμοποιούνται 80000 δείγματα για την εκπαιδεύση και τα υπόλοιπα 20000 για το validation/testing.

In [ ]:
len(dataset)

674898

#### Ερώτηση 8

>*Δείτε <a href="https://en.wikipedia.org/wiki/Cross-validation_(statistics)">εδώ</a> μεθόδους που μπορεί να χρησιμοποιούν έναντι της μεθόδου K-fold cross validation. Να αναφέρετε μερικά πλεονεκτήματα και μειονεκτήματά τους.*

Μερικές εναλλακτικές μέθοδοι έναντι του *K-fold cross validation* είναι οι εξής:<br>

<b>*Exhaustive cross-validation methods*</b>

- <b>*Leave-p-out cross-validation:*</b> Αυτή η μέθοδος χρησιμοποιεί $p$ το πλήθος παρατηρήσεις απ' το σύνολο των δεδομένων για το σύνολο επικύρωσης και τις υπόλοιπες για το σύνολο εκπαίδευσης. Η διαδικασία αυτή επαναλαμβάνεται για όλους τους πιθανούς συνδυασμούς *p* στοιχείων που μπορούν να δημιουργηθούν. Το πλεονέκτημα αυτής της μεθόδου είναι ότι θα ελεγχθεί το μοντέλο πάνω σε πολλά διαφορετικά σύνολα δεδομένων. Το μειονέκτημα της μεθόδου είναι ότι για μεγάλο πλήθος δεδομένων η μέθοδος μπορεί να μην είναι υλοποιήσιμη εξαιτίας του μεγάλου υπολογιστικού κόστους.
<br>

- <b>*Leave-one-out cross-validation:*</b> Η μέθοδος αυτή αντιστοιχεί με την *leave-p-out cross-validation* όπου $p=1$. Δηλαδή, σε κάθε επανάληψη αντί το μοντέλο να εξετάζεται σε $p>1$ το πλήθος στοιχεία εξετάζεται μόνο σε 1. Η μέθοδος αυτή απαιτεί μικρότερο αριθμό επαναλήψεων απ'ότι η *leave-one-out cross-validation*, παρ'όλα αυτά για μεγάλο το πλήθος δείγμα μπορεί να απαιτεί και αυτή αρκετό χρόνο.
<br>

<b>*Non-exhaustive cross-validation methods*</b><br>

Στις *non-exhaustive* cross-validation μεθόδους δεν χρειάζεται να υπολογιστούν όλοι οι πιθανοί συνδυασμοί που προκύπτουν απ' το σύνολο των δεδομένων. Σε αυτές τις μεθόδους ανήκει και η *K-fold cross-validation*.<br>

- <b>*Holdout method:*</b> Στη μέθοδο *Holdout* κάθε σημείο του συνόλου δεδομένων αντιστοιχίζεται σε ένα απ' τα δύο σύνολα $d_1,d_2$. Το $d_1$ αντιστοιχεί στο σύνολο εκπαίδευσης ενώ το $d_2$ στο σύνολο επικύρωσης. Συνήθως, το μέγεθος του *train set* είναι μικρότερο του συνόλου εκπαίδευσης. Ένα μειονέκτημα αυτής της μεθόδου είναι ότι μπορεί, ανάλογα με το πως θα διαμεριστεί το σύνολο των δεδομένων, να δώσει μη ρεαλιστικά αποτελέσματα.<br>

- <b>*Repeated random sub-sampling validation:*</b> Η μέθοδος αυτή είναι γνωστή και ως *Monte Carlo*. Αυτή η μέθοδος δημιουργεί τυχαία υποσύνολα του συνόλου δεδομένων καθένα απ'τα οποία χωρίζεται περαιτέρω σε σύνολο εκπαίδευσης και σύνολο επικύρωσης. Σε κάθε τέτοιο σύνολο το μοντέλο προσαρμόζεται στο σύνολο εκπαίδευσης και αξιολογείται στο σύνολο επικύρωσης. Η τελική ακρίβεια προκύπτει απ' τον μέσο όρο των επιμέρους αποτελεσμάτων. Το αποτέλεσμα αυτής της μεθόδου είναι ότι το η αναλογία του μεγέθους μεταξύ του *train* και του *test* set δεν εξαρτάται απ' τον αριθμό των επαναλήψεων. Το μειονέκτημα της μεθόδου είναι ότι μερικές παρατηρήσεις μπορεί να μην βρεθούν ποτέ στο σύνολο επικύρωσης ενώ άλλες μπορεί να βρεθούν παραπάνω από μια φορά.

#### Ερώτηση 9

>*Να αναλύσετε τα κριτήρια precision, recall και F1-score που εμφανίζονται στο classification report.*

Για τα παρακάτω υποθέτουμε ότι έχουμε ένα πρόβλημα κατηγοριοποίησης $N$ κλάσεων. Για κάθε $1\leq i\leq N$, με $\text{TP}_i,\, \text{FP}_i,\, \text{TN}_i,\, \text{FN}_{i}$ συμβολίζουμε τα *true positive, false positive, true negative* και *false negative* της κλάσης $i$, αντιστοίχως.<br>

- <b>Precision:</b> Το *precision* για μια κλάση $i$ δίνεται μέσω της σχέσης<br>

$$\text{Precision}_i = \frac{\text{TP}_i}{\text{TP}_i + \text{FP}_i},$$

<br> και εκφράζει το ποσοστό των σωστών προβλέψεων που έγιναν για την κλάση $i$ προς όλες τις προβλέψεις που έγιναν για την κλάση $i$.

- <b>Recall:</b> Το *recall* δίνεται μέσω της σχέσης<br>

$$\text{Recall}_{i} = \frac{\text{TP}_i}{\text{TP}_i + \text{FN}_i},$$

<br> και εκφράζει το ποσοστό των σωστών προβλέψεων που έγιναν για την κλάση $i$.

- <b>F1-score:</b> Το F1-score συνδυάζει συγχρόνως και το *precision* και το *recall* σε μια ποσότητα μέσω του αρμονικού μέσου<br>

$$\text{F1-score}_i = 2\cdot \frac{\text{Precision}_i \cdot \text{Recall}_i}{\text{Precision}_i + \text{Recall}_i}.$$


#### Ερώτηση 10

>*Σε ποιες κατηγορίες πετυχαίνουμε καλύτερα αποτελέσματα και σε ποιες χειρότερα; Γιατί πιστεύετε ότι παίρνουμε αυτά τα αποτελέσματα;*

Όπως βλέπουμε και από τα classification report οι κατηγορίες στις οποίες το μοντέλο πετυχαίνει καλύτερα αποτελέσματα είναι οι: Alexa, qadars, corebot, rambo, padcrypt και rovnix. Ενώ τα χειρότερα αποτελέσματα στις: dircrypt, fobben, kraken, ramnit και necurs. Τα αποτελέσματα αυτά μπορεί να οφείλονται σε διάφορους λόγους, για παράδειγμα στην περίπτωση της κατηγορίας alexa όπως βλέπουμε το support της είναι δυσανάλογο σε σχέση με τις υπόλοιπες κατηγορίες με αποτέλεσμα ο αλγόριθμος να μαθαίνει αρκετά καλά αυτή την κατηγορία.